O Algoritmo de Quine–McCluskey (ou método dos implicantes primos) é um método utilizado para minimização de funções booleanas desenvolvido por W.V. Quine e Edward J. McCluskey em 1956. O procedimento consiste em 3 etapas:


1.  Encontrar todos os implicantes primos da função;
2.  Usar esses implicantes primos num mapa de implicantes primos para encontrar os implicantes primos essenciais da função;
3.  Usar os implicante primos essenciais e se necessário alguns implicantes primos para encontrar a função minimizada.

Embora mais prático do que o mapa de Karnaugh ao lidar com mais de 4 variáveis, o algoritmo de Quine-McCluskey também possui limitações devido ao fato de o problema resolvido por ele ser NP-difícil: o tempo de execução do algoritmo de Quine-McCluskey cresce exponencialmente em relação ao número de variáveis. Pode-se mostrar que para uma função de n variáveis o limite superior no número de implicantes primos é 3n/n. Se n = 32 haverá cerca de 5.8 . 1013 implicantes primos. Portanto o algoritmo proposto neste trabalho só trata funções com no máximo 4 variáveis.



In [114]:
import string
import re
from collections import defaultdict
from itertools import combinations

In [59]:
class ValidadorSintaxe(Exception):
    def __init__(self, valor):
        self.valor = valor
        
    def __str__(self):
        return repr(self.valor)
    
class ValidadorCaracter(Exception):
    def __init__(self, valor):
        self.valor = valor
        
    def __str__(self):
        return repr(self.valor)

Verifica se a lista de símbolos dada representa a expressão lógica sintaticamente correta

In [139]:
def ChecaSintaxe(simbolosExpressao):
    
    operadores = '*|^=>'
    contadorColchetes = 0
    estado = 2
    
    for simbolo in simbolosExpressao:
        
        if estado == 1:
            
            if simbolo in operadores:
                estado = 2
                continue
                
            if simbolo == ')':
                contadorColchetes -= 1
                
                if contadorColchetes < 0:
                    return False
                
                continue
                
            return False
        
                
        elif estado == 2:
            
            if simbolo == '(':
                contadorColchetes += 1
                continue
                
            if simbolo == '!':
                continue
                
            if simbolo == ')' or simbolo in operadores:
                return False
            
            estado = 1
            
    
    return estado == 1 and contadorColchetes == 0

Converte determinada string em representação de sintaxe postfix

'0' e '1' também são tratados como variáveis neste contexto

Dividindo a expressão em lista de entradas únicas (variáveis ou operadores)

Todos os operadores considerados, exceto '!', são associativos da esquerda para a direita


In [140]:
def ConverteRpn(expressao):
    
    caracteresVariavel = string.ascii_letters + string.digits
    caracteresOperacao = '*|.>=!()'
    
    caracteresInvalidos = set([char for char in expressao
                          if char not in (caracteresVariavel + caracteresOperacao + string.whitespace)])
    
    if len(caracteresInvalidos) != 0:
        raise ValidadorCaracter('Caracters inválidos: ' + ', '.join(caracteresInvalidos))
    
    simbolos = re.findall('[' + caracteresVariavel + ']+|[' + caracteresOperacao + ']', expressao)
    
    if not ChecaSintaxe(simbolos):
        raise ValidadorSintaxe("Sintaxe inválida da expressão")
    
    associatividade = defaultdict(lambda: 'left')
    associatividade['!'] = 'right'
    
    precedencia = {'(': 6,
                  ')': 6,
                  '!': 5,
                  '*': 3,
                  '|': 2,
                  '.': 4,
                  '>': 1,
                  '=': 0}
    
    pilha = []
    fila = []
    
    for simbolo in simbolos:
        
        if simbolo not in caracteresOperacao:
            fila.append(simbolo)
            continue
            
        if simbolo == '(':
            pilha.append(simbolo)
            continue
            
        if simbolo == ')':
        
            while pilha[-1] != '(':
                fila.append(pilha.pop())
                
            pilha.pop()
            continue
            
        while (len(pilha) > 0 and
               pilha[-1] != '(' and (
               precedencia[pilha[-1]] > precedencia[simbolo] or
               (precedencia[pilha[-1]] == precedencia[simbolo] and associatividade[pilha[-1]] == 'left'))):
            
            fila.append(pilha.pop())
    
        pilha.append(simbolo)
    
    
    while len(pilha) > 0:
        fila.append(pilha.pop())
        
    return fila

Avalia o valor da expressão

In [141]:
def AvaliaValor(expressaoRpn, valores):
    
    pilha = []
    
    for simbolo in expressaoRpn:
        
        if simbolo == '=':
            pilha = pilha[:-2] + [ pilha[-2] == pilha[-1] ]
            
        elif simbolo == '>':
            pilha = pilha[:-2] + [ (not pilha[-2]) or pilha[-1] ]
            
        elif simbolo == '*':
            pilha = pilha[:-2] + [ pilha[-2] and pilha[-1] ]
            
        elif simbolo == '|':
            pilha = pilha[:-2] + [ pilha[-2] or pilha[-1] ]
            
        elif simbolo == '.':
            pilha = pilha[:-2] + [ pilha[-2] != pilha[-1] ]
            
        elif simbolo == '!':
            pilha = pilha[:-1] + [ not pilha[-1] ]
            
        elif simbolo == '1':
            pilha += [True]
            
        elif simbolo == '0':
            pilha += [False]
            
        else:
            pilha += [ valores[simbolo] ]
                
    return pilha[0]

Extraindo todas as variaveis da expressao e cria uma lista ordenada delas

Craindo uma máscara binária a partir de um número

Gera dicionário de valores para determinado conjunto de variáveis

Conta o número de '1' dígitos na representação binária do número

Encontra todos os conjuntos de valores para os quais a expressão é avaliada como True

In [137]:
def PegaVariaveis(expressaoRpn):
    return sorted(set([simbolo for simbolo in expressaoRpn if simbolo not in '*|^>=!01']))

def MascaraBinaria(numero, largura):
    return bin(numero)[2:].rjust(largura, '0')

def PegaValores(mascarBinariaNum, variaveis):
    
    resultado = {}
    valores = map(lambda x: x == '1', MascaraBinaria(mascarBinariaNum, len(variaveis)))
    
    for variavel, valor in zip(variaveis, valores):
        resultado[variavel] = valor
        
    return resultado

def PegaNumeroUm(mascarBinariaNum):
    return bin(mascarBinariaNum).count('1')

def PegaMintermos(expressaoRpn):
    
    variaveis = PegaVariaveis(expressaoRpn)
    tamanho = len(variaveis)
    print("Número de variáveis", tamanho)
    if(tamanho > 4):
      print("Programa não aceita mais de 4 variáveis")
      return 1
    limiteSuperior = 1 << tamanho
    
    mintermos = set()
    
    for mascarBinariaNum in range(limiteSuperior):
        
        valores = PegaValores(mascarBinariaNum, variaveis)
        
        if AvaliaValor(expressaoRpn, valores) == True:
            minterm_bitmask = 1 << mascarBinariaNum
            mintermos.add((minterm_bitmask, MascaraBinaria(mascarBinariaNum, tamanho)))
            
    return mintermos, variaveis

Mescla dois implicantes (por exemplo, mescla('100', '110') -> '1-0')

In [119]:
def MesclarImplicantes(implicanteA, implicanteB):
    
    mascaraA = implicanteA[1]
    mascaraB = implicanteB[1]
    
    if len([_ for bitA, bitB in zip(mascaraA, mascaraB) if bitA != bitB]) != 1:
        return None
    
    resultMascara = ""
    
    for bitA, bitB in zip(mascaraA, mascaraB):
        resultMascara += bitA if bitA == bitB else '-'
        
    return (implicanteA[0] | implicanteB[0]), resultMascara

Agrupa implicantes até que nenhuma outra combinação seja possível

In [120]:
def PegaPrimosImplicantes(mintermos):
    
    print("Mintermos: ", mintermos)
    implicantes = mintermos
    primosImplicantes = set()
    
    while True:
    
        used_implicantes = set()
        novosImplicantes = set()
        
        for implicanteA, implicanteB in combinations(implicantes, 2):

            merged = MesclarImplicantes(implicanteA, implicanteB)
                
            if merged is not None:
                novosImplicantes.add(merged)
                used_implicantes.add(implicanteA[0])
                used_implicantes.add(implicanteB[0])
        
        for imp in implicantes:
            if imp[0] not in used_implicantes:
                primosImplicantes.add(imp)
                    
        implicantes = novosImplicantes
        
        if len(implicantes) == 0: 
            break
    print("Primos implicantes", primosImplicantes)
    return primosImplicantes

Encontra o menor conjunto de principais implicantes que cobrem todos os mintermos

In [121]:
def PegaMintermosCobertos(mintermos, primosImplicantes):
    
    mascaraMintermos = 0
    for mintermo, _ in mintermos:
        mascaraMintermos |= mintermo
    
    for i in range(1, len(mintermos)+1):
        for setaPrimos in combinations(primosImplicantes, i):
            
            mascaraPrimos = 0
            for primo, _ in setaPrimos:
                mascaraPrimos |= primo
                
            if mascaraPrimos == mascaraMintermos:
                return setaPrimos
            
    return None

Convertendo implicante em string do conjunto de variáveis

Convertendo o conjunto de implicantes em string de expressão lógica

In [138]:
def ConverteString(implicant, variaveis):
    
    resultado = []
    for simbolo, variavel in zip(implicant[1], variaveis):
        if simbolo == '1':
            resultado += [variavel]
        elif simbolo == '0':
            resultado += ['!' + variavel]
            
    return ' * '.join(resultado)

def ImplicantesParaString(cobertura, variaveis):
    
    resultado = [ConverteString(implicant, variaveis) for implicant in cobertura]
    return ' | '.join(resultado)

Simplifica a expressão lógica com o algoritmo Quine-McCluskey

In [124]:
def Simplifica(expressao):
    
    expressaoRpn = ConverteRpn(expressao)
    mintermos, variaveis = PegaMintermos(expressaoRpn)
    
    if len(variaveis) == 0:
        return '1' if AvaliaValor(expressaoRpn, {}) else '0'
    
    if len(mintermos) == 0:
        return '0'
    
    if len(mintermos) == 1 << len(variaveis):
        return '1'
    
    primosImplicantes = PegaPrimosImplicantes(mintermos)
    coberturaMintermos = PegaMintermosCobertos(mintermos, primosImplicantes)
    return ImplicantesParaString(coberturaMintermos, variaveis)

#Testes

In [143]:
res = Simplifica("(!A * !B * !C * !D) | (!A * !B * !C * D) | (!A * B * !C * D) | (!A * B * C * !D) | (!A * B * C * D)")
contadorLiterais = 0
for i in range(len(res)):
  if( res[i] == "A" or res[i] == "B" or res[i] == "C" or res[i] == "D" or res[i] == "E" or res[i] == "F" or res[i] == "G" or
  res[i] == "H" or res[i] == "I" or res[i] == "J" or res[i] == "K" or res[i] == "L" or res[i] == "M" or res[i] == "N" or res[i] == "O" or res[i] == "P" or 
  res[i] == "Q" or res[i] == "R" or res[i] == "S" or res[i] == "T" or res[i] == "U" or res[i] == "V" or res[i] == "X" or res[i] == "W" or res[i] == "Y" or 
  res[i] == "Z"):
    contadorLiterais += 1

print("Resultado da simplificação: ", res)
print("Número de literais", contadorLiterais)

Número de variáveis 4
Mintermos:  {(32, '0101'), (128, '0111'), (64, '0110'), (1, '0000'), (2, '0001')}
Primos implicantes {(160, '01-1'), (192, '011-'), (34, '0-01'), (3, '000-')}
Resultado da simplificação:  !A * B * D | !A * B * C | !A * !B * !C
Número de literais 9


Algoritmo: (!A * B * D) | (!A * B * C) | (!A * !B * !C)

Karma: (!A * !B* !C) | (!A * B * C) | (!A * B * D)

**EQUIVALENTES**


In [146]:
res = Simplifica("(!A*B*!C*D)|(!A*B*C*!D)|(!A*B*C*D)|(A*!B*!C*D)|(A*!B*C*!D)|(A*!B*C*D)|(A*B*!C*D)|(A*B*C*!D)")
contadorLiterais = 0
for i in range(len(res)):
  if( res[i] == "A" or res[i] == "B" or res[i] == "C" or res[i] == "D" or res[i] == "E" or res[i] == "F" or res[i] == "G" or
  res[i] == "H" or res[i] == "I" or res[i] == "J" or res[i] == "K" or res[i] == "L" or res[i] == "M" or res[i] == "N" or res[i] == "O" or res[i] == "P" or 
  res[i] == "Q" or res[i] == "R" or res[i] == "S" or res[i] == "T" or res[i] == "U" or res[i] == "V" or res[i] == "X" or res[i] == "W" or res[i] == "Y" or 
  res[i] == "Z"):
    contadorLiterais += 1

print("Resultado da simplificação: ", res)
print("Número de literais", contadorLiterais)

Número de variáveis 4
Mintermos:  {(32, '0101'), (8192, '1101'), (1024, '1010'), (128, '0111'), (2048, '1011'), (64, '0110'), (512, '1001'), (16384, '1110')}
Primos implicantes {(3072, '101-'), (192, '011-'), (2560, '10-1'), (160, '01-1'), (16448, '-110'), (17408, '1-10'), (8224, '-101'), (8704, '1-01')}
Resultado da simplificação:  A * !B * C | !A * B * D | B * C * !D | A * !C * D
Número de literais 12


Algoritmo: (A * !B * C) | (!A * B * D) | (B * C * !D) | (A * !C * D)

Karma: (A * !B * D) | (!A * B * C) | (B * !C * D) | (A * C * !D)

**EQUIVALENTES**

In [149]:
res = Simplifica("(!A*C)|(!A*B)|(A*!B*!C)|(!B*!C*!D)")
contadorLiterais = 0
for i in range(len(res)):
  if( res[i] == "A" or res[i] == "B" or res[i] == "C" or res[i] == "D" or res[i] == "E" or res[i] == "F" or res[i] == "G" or
  res[i] == "H" or res[i] == "I" or res[i] == "J" or res[i] == "K" or res[i] == "L" or res[i] == "M" or res[i] == "N" or res[i] == "O" or res[i] == "P" or 
  res[i] == "Q" or res[i] == "R" or res[i] == "S" or res[i] == "T" or res[i] == "U" or res[i] == "V" or res[i] == "X" or res[i] == "W" or res[i] == "Y" or 
  res[i] == "Z"):
    contadorLiterais += 1

print("Resultado da simplificação: ", res)
print("Número de literais", contadorLiterais)

Número de variáveis 4
Mintermos:  {(32, '0101'), (128, '0111'), (64, '0110'), (1, '0000'), (256, '1000'), (8, '0011'), (512, '1001'), (16, '0100'), (4, '0010')}
Primos implicantes {(85, '0--0'), (240, '01--'), (768, '100-'), (204, '0-1-'), (257, '-000')}
Resultado da simplificação:  !A * !D | !A * B | A * !B * !C | !A * C
Número de literais 9


Algoritmo: (!A * !D) | (!A * B) | (A * !B * !C) | (!A * C)

Karma: (A * !B * !C) | (!A * B) | (!A * C) | (!A * !D)

**EQUIVALENTES**

In [147]:
entrada = input()
res = Simplifica(entrada)
contadorLiterais = 0
for i in range(len(res)):
  if( res[i] == "A" or res[i] == "B" or res[i] == "C" or res[i] == "D" or res[i] == "E" or res[i] == "F" or res[i] == "G" or
     res[i] == "H" or res[i] == "I" or res[i] == "J" or res[i] == "K" or res[i] == "L" or res[i] == "M" or res[i] == "N" or res[i] == "O" or res[i] == "P" or 
     res[i] == "Q" or res[i] == "R" or res[i] == "S" or res[i] == "T" or res[i] == "U" or res[i] == "V" or res[i] == "X" or res[i] == "W" or res[i] == "Y" or 
     res[i] == "Z"):
     contadorLiterais += 1

print("Resultado da simplificação: ", res)
print("Número de literais", contadorLiterais)

(!A*C)|(!A*B)|(A*!B*!C)|(!B*!C*!D)
Número de variáveis 4
Mintermos:  {(32, '0101'), (128, '0111'), (64, '0110'), (1, '0000'), (256, '1000'), (8, '0011'), (512, '1001'), (16, '0100'), (4, '0010')}
Primos implicantes {(85, '0--0'), (240, '01--'), (768, '100-'), (204, '0-1-'), (257, '-000')}
Resultado da simplificação:  !A * !D | !A * B | A * !B * !C | !A * C
Número de literais 9
